In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio
from rasterio.mask import mask
from shapely.geometry import Point, mapping

# Import Volcano Catalogue and reproject coordinates

In [2]:
volcano_data = pd.read_csv("NVP_List_3Jun.csv")

volcano_df = pd.DataFrame(volcano_data)

volcano_gdf = gpd.GeoDataFrame(
    volcano_df, 
    geometry=gpd.points_from_xy(volcano_df["LON_CENTRE"], volcano_df["LAT_CENTRE"]),
    crs="EPSG:4326"
)

volcano_gdf = volcano_gdf.to_crs("EPSG:3577")

# Import Population Density Raster Layer and confirm matching coordinates

In [5]:
density_raster = rasterio.open("Density Raster")
density_nodata = density_raster.nodata

if density_raster.crs != volcano_gdf.crs:
    volcano_gdf = volcano_gdf.to_crs(density_raster.crs)

Raster CRS: EPSG:3577
Vector CRS: EPSG:3577


# Create Buffer Zones

In [6]:
# Create buffers at 5km, 10km, 25km and 50km
volcano_gdf["buffer_5km"]  = volcano_gdf.geometry.buffer(5000)
volcano_gdf["buffer_10km"] = volcano_gdf.geometry.buffer(10000)
volcano_gdf["buffer_25km"] = volcano_gdf.geometry.buffer(25000)
volcano_gdf["buffer_50km"] = volcano_gdf.geometry.buffer(50000)

# Define three distinct zones:
# Zone 0-5:    the 5km buffer
# Zone 5-10:   the area of the 10km buffer minus the 5km buffer
# Zone 10-25:  the area of the 25km buffer minus the 10km buffer
# Zone 25-50:  the area of the 25km buffer minus the 50km buffer
volcano_gdf["zone_0_5"] = volcano_gdf["buffer_5km"]
volcano_gdf["zone_5_10"] = volcano_gdf["buffer_10km"].difference(volcano_gdf["buffer_5km"])
volcano_gdf["zone_10_25"] = volcano_gdf["buffer_25km"].difference(volcano_gdf["buffer_10km"])
volcano_gdf["zone_25_50"] = volcano_gdf["buffer_50km"].difference(volcano_gdf["buffer_25km"])


# Create a function to get buffer zone metrics

In [7]:
def get_zone_metrics(polygon, raster, nodata):
    """Return (zone_area_in_square_meters, average_density) for a given polygon."""
    if not polygon.is_valid:
        polygon = polygon.buffer(0)
    if polygon.is_empty:
        return None, None
    # Handle MultiPolygon: split into parts.
    if polygon.geom_type == 'MultiPolygon':
        shapes = [mapping(geom) for geom in polygon.geoms]
    else:
        shapes = [mapping(polygon)]
    
    try:
        out_image, out_transform = mask(dataset=raster, shapes=shapes, crop=True)
    except Exception as e:
        print("Error during masking for polygon:", e)
        return None, None

    if out_image is None or out_image.shape[0] == 0:
        return None, None
    try:
        data = out_image[0]  # single-band raster assumed.
    except IndexError as e:
        print("Error indexing the output image:", e)
        return None, None

    # Replace nodata values with NaN.
    if nodata is not None:
        data = np.where(data == nodata, np.nan, data)
    
    if np.all(np.isnan(data)):
        avg_density = np.nan
    else:
        avg_density = np.nanmean(data)
    
    # Calculate area: pixel_area * number of valid pixels.
    pixel_area_m2 = abs(out_transform.a * out_transform.e)
    valid_pixel_count = np.count_nonzero(~np.isnan(data))
    pixel_area_km2 = pixel_area_m2 / 1e6
    zone_area_km2 = valid_pixel_count * pixel_area_km2
    return zone_area_km2, avg_density


# Function to Determine Ranking Based on Average Density and Zone

In [8]:
def get_ranking_for_zone(zone, avg_density):
    """
    Return a numeric ranking for a given zone (as a string key) based on the average density.
    Zone keys: "0_5", "5_10", "10_25", "25_50"
    The numeric ranking will then be used in the overall weighted priority calculation.
    """
    if zone == "0_5":  # Within 5km buffer:
        if avg_density > 29.8:
            return 5   # Very High
        elif avg_density > 3.6:
            return 4   # High
        else:
            return 3   # Moderate
    elif zone == "5_10":  # 5 to 10km buffer:
        if avg_density > 1194.65:
            return 5   # Very High
        elif avg_density > 3.6:
            return 4 #High
        else:
            return 3   # Moderate
    elif zone == "10_25":  # 10 to 25km buffer:
        if avg_density > 1194.65:
            return 4   # High
        elif avg_density > 3.6:
            return 3   # Moderate
        else:
            return 2   # Low
    elif zone == "25_50":  #25 to 50km buffer:
        if avg_density > 29.8:
            return 3   # Moderate
        elif avg_density > 3.6:
            return 2   # Low
        else:
            return 1  #Very Low

    else:
        return None


# Loop Over Volcanoes to Compute Metrics and Weighted Priority

In [9]:
results = []
for idx, row in volcano_gdf.iterrows():
    name = row["Volcanic Centre Name"]
    
    # Get polygons for each zone.
    zone0_5_poly = row["zone_0_5"]
    zone5_10_poly = row["zone_5_10"]
    zone10_25_poly = row["zone_10_25"]
    zone25_50_poly = row["zone_25_50"]
    
    # Compute metrics for each zone.
    area_0_5, dens_0_5 = get_zone_metrics(zone0_5_poly, density_raster, density_nodata)
    area_5_10, dens_5_10 = get_zone_metrics(zone5_10_poly, density_raster, density_nodata)
    area_10_25, dens_10_25 = get_zone_metrics(zone10_25_poly, density_raster, density_nodata)
    area_25_50, dens_25_50 = get_zone_metrics(zone25_50_poly, density_raster, density_nodata)
    
    # If any metrics are None, skip this volcano.
    if (area_0_5 is None or dens_0_5 is None or 
        area_5_10 is None or dens_5_10 is None or 
        area_10_25 is None or dens_10_25 is None or
        area_25_50 is None or dens_25_50 is None):
        print(f"Skipping {name} due to missing zone metrics.")
        continue

    # Determine rankings per zone using custom rules.
    rank_0_5 = get_ranking_for_zone("0_5", dens_0_5)
    rank_5_10 = get_ranking_for_zone("5_10", dens_5_10)
    rank_10_25 = get_ranking_for_zone("10_25", dens_10_25)
    rank_25_50 = get_ranking_for_zone("25_50", dens_25_50)

    # Compute weighted priority:
    numerator = (rank_0_5   * area_0_5   * (dens_0_5**1.25) +
                 rank_5_10  * area_5_10  * (dens_5_10**1.05) +
                 rank_10_25 * area_10_25 * (dens_10_25**0.5) +
                 rank_25_50 * area_25_50 * np.cbrt(dens_25_50)) 
    denominator = (area_0_5 * (dens_0_5**1.25) +
                   area_5_10 * (dens_5_10*1.05)  +
                   area_10_25 * (dens_10_25**0.5) +
                   area_25_50 * np.cbrt(dens_25_50))
    
    if denominator == 0:
        weighted_priority = np.nan
    else:
        weighted_priority = numerator / denominator

    results.append({
        "Volcanic Centre Name": name,
        "Area_0_5_km2": area_0_5,
        "Avg_Density_0_5": dens_0_5,
        "Ranking_0_5": rank_0_5,
        "Area_5_10_km2": area_5_10,
        "Avg_Density_5_10": dens_5_10,
        "Ranking_5_10": rank_5_10,
        "Area_10_25_km2": area_10_25,
        "Avg_Density_10_25": dens_10_25,
        "Ranking_10_25": rank_10_25,
        "Area_25_50_km2": area_25_50,
        "Avg_Density_25_50": dens_25_50,
        "Ranking_25_50": rank_25_50,
        "Weighted_Priority": weighted_priority
    })



results_df = pd.DataFrame(results)


print(results_df)

     Volcanic Centre Name  Area_0_5_km2  Avg_Density_0_5  Ranking_0_5  \
0             Aitken_Hill          78.0      1675.149048            5   
1              Baird_Hill          78.0         6.372509            4   
2    Bald_Hill_Blakeville          81.0         2.352749            3   
3    Bald_Hill_Daylesford          78.0        52.879227            5   
4     Bald_Hill_Edgecombe          79.0        27.671961            4   
..                    ...           ...              ...          ...   
411               Mt_Muir          77.0         5.610287            4   
412           Mt_Muirhead          79.0         6.808551            4   
413             Mt_Schank          78.0         2.794237            3   
414              Mt_Watch          78.0         0.853093            3   
415     The_Bluff_Glencoe          78.0         1.659541            3   

     Area_5_10_km2  Avg_Density_5_10  Ranking_5_10  Area_10_25_km2  \
0            234.0        504.892456             4   

# Define a function to assign ranking labels and add to Dataframe

In [10]:
def assign_ranking_label(val):
    if val < 1.5:
        return "Very Low"
    elif 1.5 <= val < 2.5:
        return "Low"
    elif 2.5 <= val < 3.5:
        return "Moderate"
    elif 3.5 <= val < 4.5:
        return "High"
    else:  # val >= 4.5
        return "Very High"

results_df["Ranking_Label"] = results_df["Weighted_Priority"].apply(assign_ranking_label)



In [11]:
# Merge the coordinate columns into results_df
results_df = results_df.merge(
    volcano_gdf[["Volcanic Centre Name", "LAT_CENTRE", "LON_CENTRE"]],
    on="Volcanic Centre Name",
    how="left"
)

# Define the desired order for the first three columns
desired_order = ["Volcanic Centre Name", "LAT_CENTRE", "LON_CENTRE"]

# Get the rest of the columns that are not in the desired order
other_cols = [col for col in results_df.columns if col not in desired_order]

# Reorder the DataFrame columns
results_df = results_df[desired_order + other_cols]


# Optionally, rename them for clarity:
results_df = results_df.rename(columns={"LAT_CENTRE": "Latitude", "LON_CENTRE": "Longitude"})

results_df.to_csv("NVP_PDMatrix_Priority.csv", index=False)


print(results_df)

     Volcanic Centre Name  Latitude  Longitude  Area_0_5_km2  Avg_Density_0_5  \
0             Aitken_Hill  -37.6090   144.9043          78.0      1675.149048   
1              Baird_Hill  -37.3915   143.7341          78.0         6.372509   
2    Bald_Hill_Blakeville  -37.5277   144.2105          81.0         2.352749   
3    Bald_Hill_Daylesford  -37.3285   144.1100          78.0        52.879227   
4     Bald_Hill_Edgecombe  -37.1987   144.4557          79.0        27.671961   
..                    ...       ...        ...           ...              ...   
413               Mt_Muir  -37.5493   140.4568          77.0         5.610287   
414           Mt_Muirhead  -37.5587   140.4044          79.0         6.808551   
415             Mt_Schank  -37.9401   140.7362          78.0         2.794237   
416              Mt_Watch  -37.6916   140.5250          78.0         0.853093   
417     The_Bluff_Glencoe  -37.7200   140.5645          78.0         1.659541   

     Ranking_0_5  Area_5_10